In [1]:
from pymongo import MongoClient
import os
import sys
import csv
import glob
import calendar
import cPickle as pickle
import pandas as pd
from collections import defaultdict
from datetime import datetime
import re
import numpy as np
from dateutil import relativedelta
import plotly
import plotly.graph_objs as go

csv.field_size_limit(sys.maxsize)

131072

In [33]:
activity_dir = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/random_assignment/'
filename = 'ansible~ansible_kmeans_7_random.pickle'
label_dict = pickle.load(open(os.path.join(activity_dir, filename), 'rb'))
rev_dict = {}
for actor,cluster in label_dict.iteritems():
    if cluster not in rev_dict:
        rev_dict[cluster] = set()
    rev_dict[cluster].add(actor)

In [40]:
label_dict['abadger']

5

In [3]:
activity_dir = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/random_assignment/temp/'
client = MongoClient("mongodb://127.0.0.1:27017")
project_events = client["pypi-v6"].project_events
issue_events = client["pypi-v6"].issue_events
burst_dict_pickle = '/usr2/scratch/sschoudh/kdd_v6/HMM/results/correct/hmm_v6_daily_bursts_correct_filtered_states.pickle'
burst_dict = pickle.load(open(burst_dict_pickle, 'rb'))

def ConvertToDate(str_time, str_format):
    return datetime.strptime(str_time, str_format).date()

for filename in os.listdir(activity_dir):
    if '.pickle' not in filename:
        continue
    print "Processing for file = ", filename
    label_dict = pickle.load(open(os.path.join(activity_dir, filename), 'rb'))
    parts = filename.split('_kmeans_')
    project = parts[0]
    num_clusters = int(parts[1].strip('_random.pickle').strip())
    project_owner = project.split('~')[0].strip()
    project_name = project.split('~')[1].strip()
    rev_dict = {}
    other_dict = {}
    for actor,cluster in label_dict.iteritems():
        if cluster not in rev_dict:
            rev_dict[cluster] = set()
        rev_dict[cluster].add(actor)

    for cluster, actor_set in rev_dict.iteritems():
        actor_set = list(actor_set)
        match_list = [re.compile('^' + re.escape(str(x)) + '$', re.IGNORECASE) for x in actor_set]
        ievents = issue_events.find({"project_owner": re.compile(project_owner, re.IGNORECASE),
                                     "project_name": re.compile(project_name, re.IGNORECASE), 
                                     "actor": {"$in" : match_list }})
        pevents = project_events.find({"project_owner": re.compile(project_owner, re.IGNORECASE),
                                     "project_name": re.compile(project_name, re.IGNORECASE), 
                                     "actor": {"$in" : match_list }})
        for irow in ievents:
            t = irow["time"].date()
            if t not in other_dict:
                other_dict[t] = defaultdict(int)
            other_dict[t][cluster] += 1
        for prow in pevents:
            t = prow["time"].date()
            if t not in other_dict:
                other_dict[t] = defaultdict(int)
            other_dict[t][cluster] += 1
        
        #Plot with boundaries
        project = project_owner + '~' + project_name
        project_bursts = burst_dict[project]
        boundaries = set()
        for cur_burst in project_bursts:
            parts = cur_burst.split("-")
            start = ConvertToDate(parts[0], '%Y/%m/%d')
            end = ConvertToDate(parts[1], '%Y/%m/%d')
            boundaries.add(start)
            boundaries.add(end)

        days = other_dict.keys()
        days = sorted(days)
        data = []
        prefix = "cluster_"
        cur = days[0]
        end = days[-1]
        series = []
        while cur <= end:
            series.append(cur)
            cur += relativedelta.relativedelta(days=1)

        maxval = 0
        for cluster in range(num_clusters):
            vals = []
            for day in series:
                if day in days:
                    vals.append(other_dict[day][cluster])
                    maxval = max(maxval, other_dict[day][cluster])
                else:
                    vals.append(0)
            legend = prefix + str(cluster)
            #print len(vals)
            data.append(go.Bar(x=series, y=vals, name=legend))

        # Add burst boundaries to data
        vals = []
        for day in series:
            if day in boundaries:
                vals.append(maxval+5)
            else:
                vals.append(0)
        legend = 'burst_boundaries'
        data.append(go.Scatter(x=series, y=vals, name=legend, line = dict(
                width = 0.7,
                dash = 'dot')))
        out_name = filename.replace('.pickle', '_random_with_boundaries.html')
        out_file = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/' + out_name
        layout = go.Layout(barmode='stack')
        plotly.offline.plot({"data": data, "layout": layout},
                        filename=out_file, auto_open=False)


Processing for file =  ros~rosdistro_kmeans_5_random.pickle


In [14]:
client = MongoClient("mongodb://127.0.0.1:27017")
project_events = client["pypi-v6"].project_events
issue_events = client["pypi-v6"].issue_events
cluster_label_pickle = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/bokeh~bokeh_kmeans_4.pickle'
label_dict = pickle.load(open(cluster_label_pickle, 'rb'))
project_name = 'bokeh'
project_owner = 'bokeh'
rev_dict = {}
other_dict = {}
for actor,cluster in label_dict.iteritems():
    if cluster not in rev_dict:
        rev_dict[cluster] = set()
    rev_dict[cluster].add(actor)

for cluster, actor_set in rev_dict.iteritems():
    actor_set = list(actor_set)
    match_list = [re.compile('^' + re.escape(str(x)) + '$', re.IGNORECASE) for x in actor_set]
    ievents = issue_events.find({"project_owner": re.compile(project_owner, re.IGNORECASE),
                                 "project_name": re.compile(project_name, re.IGNORECASE), 
                                 "actor": {"$in" : match_list }})
    pevents = project_events.find({"project_owner": re.compile(project_owner, re.IGNORECASE),
                                 "project_name": re.compile(project_name, re.IGNORECASE), 
                                 "actor": {"$in" : match_list }})
#     print "issue events = ", ievents.count()
#     print "project events = ", pevents.count()
    for irow in ievents:
        t = irow["time"].date()
        if t not in other_dict:
            other_dict[t] = defaultdict(int)
        other_dict[t][cluster] += 1
    for prow in pevents:
        t = prow["time"].date()
        if t not in other_dict:
            other_dict[t] = defaultdict(int)
        other_dict[t][cluster] += 1
        

In [5]:
cluster_label_pickle

'/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/ansible~ansible_kmeans_12.pickle'

In [17]:
from dateutil import relativedelta
from datetime import datetime
import plotly
import plotly.graph_objs as go

days = other_dict.keys()
days = sorted(days)
num_clusters = 4
data = []
prefix = "cluster_"
cur = days[0]
end = days[-1]
series = []
while cur <= end:
    series.append(cur)
    cur += relativedelta.relativedelta(days=1)

for cluster in range(num_clusters):
    vals = []
    for day in series:
        if day in days:
            vals.append(other_dict[day][cluster])
        else:
            vals.append(0)
    legend = prefix + str(cluster)
    #print len(vals)
    data.append(go.Bar(x=series, y=vals, name=legend))

out_file = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/bokeh~bokeh_kmeans_4.html'
layout = go.Layout(barmode='stack')
plotly.offline.plot({"data": data, "layout": layout},
                filename=out_file, auto_open=False)


'file:///usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/bokeh~bokeh_kmeans_4.html'

In [26]:
'''
Plot everything with the segment boundaries
'''

from dateutil import relativedelta
from datetime import datetime
import plotly
import plotly.graph_objs as go

def ConvertToDate(str_time, str_format):
    return datetime.strptime(str_time, str_format).date()

burst_dict_pickle = '/usr2/scratch/sschoudh/kdd_v6/HMM/results/correct/hmm_v6_daily_bursts_correct_filtered_states.pickle'
burst_dict = pickle.load(open(burst_dict_pickle, 'rb'))

project = 'bokeh~bokeh'
project_bursts = burst_dict[project]
boundaries = set()
for cur_burst in project_bursts:
    parts = cur_burst.split("-")
    start = ConvertToDate(parts[0], '%Y/%m/%d')
    end = ConvertToDate(parts[1], '%Y/%m/%d')
    boundaries.add(start)
    boundaries.add(end)
        
days = other_dict.keys()
days = sorted(days)
num_clusters = 4
data = []
prefix = "cluster_"
cur = days[0]
end = days[-1]
series = []
while cur <= end:
    series.append(cur)
    cur += relativedelta.relativedelta(days=1)

maxval = 0
for cluster in range(num_clusters):
    vals = []
    for day in series:
        if day in days:
            vals.append(other_dict[day][cluster])
            maxval = max(maxval, other_dict[day][cluster])
        else:
            vals.append(0)
    legend = prefix + str(cluster)
    #print len(vals)
    data.append(go.Bar(x=series, y=vals, name=legend))

# Add burst boundaries to data
vals = []
for day in series:
    if day in boundaries:
        vals.append(maxval+5)
    else:
        vals.append(0)
legend = 'burst_boundaries'
data.append(go.Scatter(x=series, y=vals, name=legend, line = dict(
        width = 0.7,
        dash = 'dot')))

out_file = '/usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/bokeh~bokeh_kmeans_4_boundaries.html'
layout = go.Layout(barmode='stack')
plotly.offline.plot({"data": data, "layout": layout},
                filename=out_file, auto_open=False)

'file:///usr2/scratch/sschoudh/kdd_v6/congruence/cluster_issues/cluster_activities/bokeh~bokeh_kmeans_4_boundaries.html'

## 